Importing the Dependencies




In [ ]:

# Install required packages
!pip install transformers accelerate gradio gradio_client

import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
from gradio_client import Client

# Set your Hugging Face token
os.environ["HF_TOKEN"] = 'YOUR_HUGGINGFACE_TOKEN'

# Load model and tokenizer
model_name = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.environ["HF_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=os.environ["HF_TOKEN"],
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

  Using cached gradio_client-1.8.0-py3-none-any.whl.metadata (7.1 kB)
Using cached gradio_client-1.8.0-py3-none-any.whl (322 kB)
  Attempting uninstall: gradio_client
    Found existing installation: gradio_client 1.0.1
    Uninstalling gradio_client-1.0.1:
      Successfully uninstalled gradio_client-1.0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Invoke the LLM

In [7]:
# Test the model
input_text = "What is the meaning of life!!!"
input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
response = model.generate(**input_ids, max_new_tokens=512)
output = tokenizer.decode(response[0], skip_special_tokens=True)
print(output)

What is the meaning of life!!!

The meaning of life is a profound question that has captivated philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer, but there are many different perspectives on this complex and multifaceted topic.

**Some common perspectives on the meaning of life include:**

* **Existentialism:** Existentialists believe that life is inherently meaningless and that we are free to choose our own meaning.
* **Nihilism:** Nihilists believe that life is devoid of meaning and purpose and that there is no point in pursuing anything.
* **Stoicism:** Stoics believe that life is what it is and that we should accept what we cannot control and focus on what we can.
* **Religion:** Many religions offer a framework for understanding the meaning of life, such as the pursuit of religious knowledge, the service to others, or the search for spiritual enlightenment.
* **Personalism:** Personalists believe that the meaning of life is d

Deploy LLM with Gradio as API

In [ ]:
# Create Gradio interface
def get_response(input_data):
    input_data = json.loads(input_data)
    input_text = input_data["query"]

    input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
    response = model.generate(**input_ids, max_new_tokens=512)
    output = tokenizer.decode(response[0], skip_special_tokens=True)

    return json.dumps({"output": output})

demo = gr.Interface(fn=get_response, inputs="json", outputs="json")
url = demo.queue().launch(share=True, debug=True, show_error=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e20a96712c8d58052a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Invoke Gradio API

In [ ]:
# Wait a moment for the Gradio server to start
import time
time.sleep(5)

# Test the client
input_data = {"query": "What is a LLM?"}
input_data_json = json.dumps(input_data)

client = Client(url)
result = client.predict(input_data_json, api_name="/predict")
print("Client result:", result)

# Alternatively, you can use the result like this:
result_dict = json.loads(result)
print("Model output:", result_dict["output"])